In [3]:
import numpy as np

# Reading Data
with open("1268-0.txt", "r", encoding="utf8") as fp:
    text = fp.read()

start_indx = text.find("THE MYSTERIOUS ISLAND")
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)

print('Total Length: ', len(text))
print('Unique Characters:', len(char_set))

Total Length:  1130711
Unique Characters: 85


In [7]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)
text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32
)

print('Text encoded shape:', text_encoded.shape)

print(text[:15], '== Encoding ==>', text_encoded[:15])
print(text_encoded[15:21], '== Reverse ==>',''.join(char_array[text_encoded[15:21]]))

Text encoded shape: (1130711,)
THE MYSTERIOUS  == Encoding ==> [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1]
[37 47 40 29 42 32] == Reverse ==> ISLAND


In [10]:
for ex in text_encoded[:5]:
    print('{} -> {}'.format(ex, char_array[ex]))

48 -> T
36 -> H
33 -> E
1 ->  
41 -> M
